# Unsupervised Deep Learning For Food Group Tags

### Purpose
The point of this notebook is to get a list of words highly associated with each food description. The associated words can be used to pre-populate 'tags' users might add to foods in the meal recommender system. Vector space modeling is the leading model to be used. 

### Part 1. Cleaning Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Can replace this with an S3 reference
df = pd.read_csv('C:/Users/J/Desktop/Businesses/Meal_Maker/Scraped_Data/combined_nutrition_small/nutrition_sm_2018_3_15_processed_comma.csv', delimiter = ',',encoding='ISO-8859-1')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Throughout this analysis, the format of food descriptions will all depend on the source of the data. Printing the unique values of the source column below will be a useful reference. 

In [6]:
df.source.unique()

array(['diet_facts_restaurants', 'diet_facts_brands', '700', 'fat_secret',
       'usda_raw_ingred', 'fat_secret_all_search', 'fat_secret_recipes',
       'usda_branded'], dtype=object)

In [3]:
df = df[df.source!='700']

Depending on its source, the format of food descriptions will follow some specific patterns. Before getting into Word2Vec, want to normalize the food description column to only include a description of the food. 

### USDA Branded Products 

In [6]:
df.food_description[df.source=='usda_branded'][0:5]

235133                         AARDVARK HABENERO HOT SAUCE,
235134            AARON'S BEST, OVEN ROASTED TURKEY BREAST,
235135    A&B AMERICAN STYLE, MORE HEAT SMALL BATCH PEPP...
235136            A&B AMERICAN STYLE, ORGANIC PEPPER SAUCE,
235137            A&B AMERICAN STYLE, PEPPER SAUCE, GARLIC,
Name: food_description, dtype: object

These descriptions often contain the brand of the item, which is going to be less relevant to the meaning wanted out of the word vector exercise

In [7]:
df[['food_description', 'brand']][df.source=='usda_branded'][0:15]

,food_description,brand
235133,"AARDVARK HABENERO HOT SAUCE,",Secret Aardvark Trading Company
235134,"AARON'S BEST, OVEN ROASTED TURKEY BREAST,",Agri Star Meat & Poultry LLC
235135,"A&B AMERICAN STYLE, MORE HEAT SMALL BATCH PEPP...",A & B AMERICAN STYLE LLC
235136,"A&B AMERICAN STYLE, ORGANIC PEPPER SAUCE,",A & B AMERICAN STYLE LLC
235137,"A&B AMERICAN STYLE, PEPPER SAUCE, GARLIC,",Namaste Foods
235138,"A&B AMERICAN STYLE, SMALL BATCH PEPPER SAUCE, ...",A & B AMERICAN STYLE LLC
235139,"A. BAUER'S, PREPARED MUSTARD,",August Bauer's Sons Inc.
235140,"ABBA-ZABA, SNACK SIZE BITES CANDY,","Annabelle Candy Co., Inc."
235141,"ABBA-ZABA'S, TAFFY, WILD STRAWBERRY, SOUR,","Annabelle Candy Co., Inc."
235142,"ABBEY FARM, RHUBARB & GINGER PRESERVE,",Bewley Irish Imports


Unhelpful! Some brands are nicely embedded in the food description, seperated by a comma. Others contain only a part of the brand name in the product description (such as Secret Aardvark Trading Company's Aardvark Habenero Hot Sauce). But a fuzzy match on the brand name within a food description would likely also strip away helpful words like 'Candy' or 'Beverage'. 

So ultimately it doesn't look like there are helpful transformations for this subset

## Part 2: Vector Space Modeling

In [8]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Creating an baseline Word2Vec model with no negative sampling

In [12]:
text_8_path = 'C:/Users/J/Desktop/Businesses/Meal_Maker/Food Classifications/text8/text8'

In [13]:
text_8 = gensim.models.word2vec.Text8Corpus(text_8_path)

In [14]:
food_names = gensim.models.Word2Vec(text_8)

In [15]:
food_names.most_similar('eggs')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('seeds', 0.8733841180801392),
 ('flowers', 0.8649996519088745),
 ('goats', 0.8307944536209106),
 ('beans', 0.8189063668251038),
 ('fruit', 0.818173885345459),
 ('vegetables', 0.8105072379112244),
 ('larvae', 0.8049867749214172),
 ('fruits', 0.8022655248641968),
 ('bees', 0.7894999980926514),
 ('skins', 0.7847944498062134)]

In [16]:
food_names.most_similar('apple')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('macintosh', 0.7894327640533447),
 ('amiga', 0.7420880794525146),
 ('intel', 0.7306458353996277),
 ('ibm', 0.7269545793533325),
 ('amd', 0.690711259841919),
 ('atari', 0.6872498989105225),
 ('pc', 0.679271936416626),
 ('nintendo', 0.674188494682312),
 ('hypercard', 0.6720134615898132),
 ('microsoft', 0.6710932850837708)]

In [17]:
food_names.most_similar('apple pie')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


KeyError: "word 'apple pie' not in vocabulary"

In [23]:
df['food_description'][0]

'Cilantro Lime Dressing'

Iterating to See How Many Raw Descriptions Are Supported In the Text8 Corpus

In [26]:
food_vec = []
results_holder = []
for i in range(len(df)):
    try:
        word_vec = food_names[df['food_description'][1]]
        food = df['food_description'][1]
        food_vec = food_vec.append(food),
        results_holder = results_holder.append(word_vec[0:10])
    except:
        pass

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
print(len(food_vec))

Two learnings here. First and foremost, this corpus is not specific enough for the exercise. 'Apple' should be returning 'fruit' and 'snack' but not 'macintosh' and 'microsoft'. Even if I had been able to nicely process my food descriptions cleanly, the results from this corpus still leave much to be desired.

Second, the model (and corpus) needs to support phrases. Entering 'apple pie' returned no results even though there is almost certainly an apple pie Wikipedia entry in these results. 

Phrases can be supported by using Negative Sampling the final network layer rather than heirarchical softmax. We can test simple implementation by retraining on the text_8 corpus, using negative sampling with 5 noise words included.

In [36]:
food_phrases = gensim.models.Word2Vec(text_8, hs=0, negative = 2)

In [23]:
food_names.most_similar('oz')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('wizard', 0.7509090900421143),
 ('carol', 0.702263593673706),
 ('judy', 0.6979846954345703),
 ('betty', 0.6864558458328247),
 ('biopic', 0.6815564632415771),
 ('potter', 0.6814549565315247),
 ('gloria', 0.680546760559082),
 ('doc', 0.679688572883606),
 ('remake', 0.6795153021812439),
 ('mister', 0.6771171689033508)]

## Part3: Serving Sizes

A human meal planner can look at a serving size '1 can' and decide 'I can eat half a can now, and half a can later'. A machine can perform at least a somewhat similar decision if and only if it can understand that '1' is the amount and 'can' is the unit. This exercise will attempt to use text processing to seperate values and units, and at best to normalize units to the same type (i.e. 'oz' and 'ounces'). 

To start with we investigate a sample of the 'serving size' column below.

In [9]:
import nltk

In [10]:
df.serving_size_raw.head()

0    1 egg cream
1    1 egg cream
2        1 order
3        1 order
4        1 order
Name: serving_size_raw, dtype: object

So egg cream is probably not going to be a standard unit of measurement. But is there a way to recognize when a unit stands for '1 item'?

On the other hand 'order' should be a word often repeated. We can start with word frequency as a good indication of true 'units' vs descriptions of items. 

Before getting a bag of words of the serving size units, stripping away the values would be helpful

In [9]:
df['ss_numbers_raw'] = ''
for i in range((len(df)-1)):
    line_numbers = []
    line = df.iloc[i,6]
    if type(line) == str:
        for t in line.split():
            try:
                line_numbers.append(float(t))   
            except ValueError:
                pass
        df['ss_numbers_raw'][i]= line_numbers
    else:
        df['ss_numbers_raw'][i]=line

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
df[['ss_numbers_raw', 'serving_size_raw']][10000:10010]

,ss_numbers_raw,serving_size_raw
10000,"[8.0, 240.0]",8 fl oz 240 mL
10001,"[12.0, 355.0]",12 fl oz 355 mL
10002,"[1.0, 12.0, 355.0]",1 can 12 fl oz 355 mL
10003,"[8.0, 240.0]",8 fl oz 240 mL
10004,"[8.0, 240.0]",8 fl oz 240 mL
10005,"[1.0, 12.0, 355.0]",1 can 12 fl oz 355 mL
10006,"[8.0, 240.0]",2/5 bottle 8 fl oz 240 mL
10007,"[1.0, 12.0, 355.0]",1 can 12 fl oz 355 mL
10008,"[8.0, 240.0]",8 fl oz 240 mL
10009,"[8.0, 240.0]",8 fl oz 240 mL


In [3]:
df['ss_strings_raw'] = ''
for i in range(len(df.serving_size_raw)):
    line_strings = []
    line = df.iloc[i,6]
    if type(line) == str:
        for t in line.split():
            try:
                float(t)
            except ValueError:
                line_strings.append(t)
        df['ss_strings_raw'][i]= line_strings
    else:
        df['ss_strings_raw'][i]=''

NameError: name 'df' is not defined

In [21]:
df[['ss_strings_raw', 'serving_size_raw']][10000:10010]

,ss_numbers_raw,serving_size_raw
10000,"[fl, oz, mL]",8 fl oz 240 mL
10001,"[fl, oz, mL]",12 fl oz 355 mL
10002,"[can, fl, oz, mL]",1 can 12 fl oz 355 mL
10003,"[fl, oz, mL]",8 fl oz 240 mL
10004,"[fl, oz, mL]",8 fl oz 240 mL
10005,"[can, fl, oz, mL]",1 can 12 fl oz 355 mL
10006,"[2/5, bottle, fl, oz, mL]",2/5 bottle 8 fl oz 240 mL
10007,"[can, fl, oz, mL]",1 can 12 fl oz 355 mL
10008,"[fl, oz, mL]",8 fl oz 240 mL
10009,"[fl, oz, mL]",8 fl oz 240 mL


In [38]:
df.columns.values

array(['food_key', 'food_description', 'brand', 'food_type_grp', 'source',
       'ingredients_list', 'serving_size_raw', 'serving_size_val',
       'serving_size_unit', 'calories', 'protein_g', 'fat_g',
       'saturated_fat_g', 'carb_g', 'fiber_g', 'sugar_g', 'sodium_mg',
       'cholesterol_mg', 'calcium_mg', 'iron_mg', 'vit_a_mcg', 'vit_c_mg',
       'ss_numbers_raw'], dtype=object)

In [ ]:
# First 
for i in range((len(df.serving_size_raw)-1)):
    line_strings = []
    line_floats = []
    line = df.iloc[i,6]
    if type(line) == str:
        for t in line.split():
            try:
                float(t)
                line_floats.append(t)
            except ValueError:
                line_strings.append(t)
        if len(line_floats) > 1:
            text1 = line[(line.find(line_floats[0])+1):(line.find(line_floats[1])-1)].strip()
            df.serving_size_val.iloc[i] = float(line_floats[0])
            df.serving_size_unit.iloc[i] = text1
        elif len(line_floats) == 1:`
            
            df.serving_size_val.iloc[i] = float(line_floats[0])
            df.serving_size_unit.iloc[i] = line[(line.find(line_floats[0])+1):len(line)].strip()
        else:
            df.serving_size_unit.iloc[i] = line
    else:
        df.serving_size_val.iloc[i] = line

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
df.serving_size_unit.iloc[1]

'egg cream'

In [9]:
type(line_floats[0])

float

In [11]:
test = df.iloc[10004,6]

In [36]:
line_strings = []
line_floats = []
for t in test.split():
    try:
        float(t)
        line_floats.append(t)
    except ValueError:
        line_strings.append(t)
if len(line_floats) > 1:
    text1 = test[(test.find(line_floats[0])+1):(test.find(line_floats[1])-1)].strip()

In [37]:
text1

'fl oz'

In [ ]:
df.to_csv('C:/Users/J/Desktop/Businesses/Meal_Maker/Scrape d_Data/combined_nutrition_small/nutrition_sm_2018_3_16_processed.csv', delimiter = ',',encoding='ISO-8859-1')